In [ ]:
let parseLine (line : string) = 
    line.Split(" -> ")
    |> Array.map (
        fun coords -> 
            let xy = coords.Split(',', 2)
            int xy[0], int xy[1]
            )

let steps a b =
    if a < b then
        [ a .. b ]
    else 
        [ b .. a ]

let connect (x1,y1) (x2,y2) =
    if x1 = x2 then
        steps y1 y2 |> Seq.map (fun y -> x1,y)
    else
        steps x1 x2 |> Seq.map (fun x -> x,y1)

let linePoints coords =
    coords
    |> Array.pairwise
    |> Seq.collect (fun (a, b) -> connect a b)            
    |> Set.ofSeq

type Cell = | Rock | Sand    

let parseBoard lines =
    lines
    |> Seq.map parseLine
    |> Seq.map linePoints
    |> Set.unionMany
    |> Seq.map (fun pos -> pos, Rock)
    |> Map.ofSeq

let rec trackFall outOfBounds (x,y) (board : Map<int*int, Cell>) =
    if y = outOfBounds then
        None
    else
        match board |> Map.tryFind (x, y+1) with
        | None ->
            trackFall outOfBounds (x, y+1) board
        | Some _ ->
            match board |> Map.tryFind (x-1, y+1) with
            | None -> trackFall outOfBounds (x-1, y+1) board
            | _ ->
                match board |> Map.tryFind (x+1, y+1) with
                | None -> trackFall outOfBounds (x+1, y+1) board
                | _ -> Some ((), board |> Map.add (x,y) Sand)

let countSand board = 
    let outOfBounds = 1 + (board |> Map.keys |> Seq.map snd |> Seq.max)

    board
    |> Seq.unfold (fun b -> trackFall outOfBounds (500,0) b)
    |> Seq.length



In [ ]:
#r "nuget: FsUnit"

open FsUnitTyped

let testBoard = 
    parseBoard [ "498,4 -> 498,6 -> 496,6"; "503,4 -> 502,4 -> 502,9 -> 494,9" ]

testBoard |> countSand |> shouldEqual 24

In [ ]:
open System.IO

let sourcePath = Path.Combine(__SOURCE_DIRECTORY__, "input_14.txt")
let board = 
    File.ReadAllLines(sourcePath)
    |> parseBoard

let sand = countSand board

In [ ]:
printfn "Sand count %d" sand

## Part 2

In [ ]:
let rec trackFall2 getItem (x,y) (board : Map<int*int, Cell>) =
    match getItem (x, y+1) with
    | None ->
        trackFall2 getItem (x, y+1) board
    | Some _ ->
        match getItem (x-1, y+1) with
        | None -> trackFall2 getItem (x-1, y+1) board
        | _ ->
            match getItem (x+1, y+1) with
            | None -> trackFall2 getItem (x+1, y+1) board
            | _ -> board |> Map.add (x,y) Sand

let countSand2 board = 
    let outOfBounds = 2 + (board |> Map.keys |> Seq.map snd |> Seq.max)
    let getItem board (x, y) = 
        if y = outOfBounds then 
            Some Rock
        else 
            board |> Map.tryFind (x, y)

    let fallen =
        board
        |> Seq.unfold (
            fun b -> 
                let newB = trackFall2 (getItem b) (500,0) b
                match newB |> Map.tryFind (500,0) with
                | Some Sand -> None
                | _ -> Some ((), newB)
            )
        |> Seq.length
    
    fallen + 1

In [ ]:
countSand2 testBoard |> shouldEqual 93

In [ ]:
let result = countSand2 board
printf "%d sand at rest" result